In [ ]:
from langchain_ollama import OllamaEmbeddings
from pathlib import Path
from rag import textbookRAG

In [32]:
from test_metadata import metadata


In [33]:
book_dir = Path("../utilities/book")

In [34]:
rag = textbookRAG(metadata=metadata, book_dir=book_dir)
# rag.extract_from_metadata()

In [35]:
rag.text_splitter._chunk_size

1300

In [36]:
len(rag.vector_store.get()["ids"])

576

In [37]:
from langchain_ollama.llms import OllamaLLM
from langchain_core.output_parsers import StrOutputParser


In [132]:
student_input = "Investigate diffusion in a non-living system (agar jelly)"

In [208]:
resultrag = rag.search(student_input, subject = "Biology", chapter=1)

In [218]:
print(resultrag)

- Understand the processes of diffusion, osmosis and active transport by which substances move into and out of cells
- Understand how factors affect the rate of movement of substances into and out of cells
- Investigate diffusion in a non-living system (agar jelly)
- Describe the levels of organisation within organisms – organelles, cells, tissues, organ systems
Uncorrected proof, all content subject to change at publisher discretion. Not for resale, circulation or distribution in whole or in part. ©Pearson 2019

## ACTIVITY 4

> Safety note: Wear eye protection and avoid all skin contact with the acid and the dyed agar blocks.

### ▼ PRACTICAL: DEMONSTRATION OF DIFFUSION IN A JELLY

Agar is a jelly that is used for growing cultures of bacteria. It has a consistency similar to the cytoplasm of a cell. Like cytoplasm, it has a high water content. Agar can be used to show how substances diffuse through a cell.

This demonstration uses the reaction between hydrochloric acid and potassium 

In [145]:
!ollama rm qna

deleted 'qna'


In [ ]:
!ollama create qna -f ./others/Modelfile

In [212]:
!ollama list

NAME                                        ID              SIZE      MODIFIED       
qna:latest                                  8f4a73ac1b7c    4.9 GB    31 minutes ago    
llama3.1:latest                             46e0c10c039e    4.9 GB    46 hours ago      
bge-m3:latest                               790764642607    1.2 GB    46 hours ago      
eurecom-ds/phi-3-mini-4k-socratic:latest    4e692a71bc19    7.6 GB    2 months ago      
llama3.2:1b                                 baf6a787fdff    1.3 GB    2 months ago      
llama2:latest                               78e26419b446    3.8 GB    2 months ago      
llama3.2:latest                             a80c4f17acd5    2.0 GB    4 months ago      


In [ ]:
!ollama show --modelfile qna

In [220]:
from pydantic import BaseModel
from typing import List

# Define the QnA pair structure
class QnaPair(BaseModel):
    question: str
    answer: str
    source: str

# Define a container for multiple QnA pairs
class QnaPairs(BaseModel):
    qa_pairs: List[QnaPair]

# Update your prompt template to explicitly request multiple pairs
llm_prompt_generate_question = """
Your task is to create as many as possible distinct question and answer pairs that will fulfill this learning objective: {student_input}

The question must begin with one of the following IGCSE command words: 'analyse', 'assess', 'calculate', 'comment', 'compare', 'consider', 'contrast', 'define', 'describe', 'develop', 'discuss', 'evaluate', 'explain', 'give', 'identify', 'justify', 'outline', 'predict', 'state', 'suggest', 'summarise'.

Your questions should be answerable with specific pieces of factual information from this context:
{context}

You should include which statement from the text file that you use to support each answer.

Respond only with valid JSON. Do not write an introduction or summary. Your response should be in this format:
{{
  "qa_pairs": [
    {{
      "question": (your question),
      "answer": (your answer to the question),
      "source": (the statement from the context that supports your answer)
    }},
    {{
      "question": (your question),
      "answer": (your answer to the question),
      "source": (the statement from the context that supports your answer)
    }},
    // Additional pairs...
  ]
}}
"""

llm_prompt = llm_prompt_generate_question.format(student_input=student_input, context=resultrag)

# Update your model invocation
model = OllamaLLM(model="qna", format="json")
response = model.invoke(llm_prompt, format=QnaPairs.model_json_schema())

# Parse the response
qna_pairs = QnaPairs.model_validate_json(response)

# Access the results
for pair in qna_pairs.qa_pairs:
    print(f"Q: {pair.question}")
    print(f"A: {pair.answer}")
    print(f"Source: {pair.source}")
    print("-" * 50)

Q: What is agar jelly used for in this demonstration?
A: Agar jelly is used to show how substances diffuse through a cell.
Source: Agar can be used to show how substances diffuse through a cell.
--------------------------------------------------
Q: What happens when hydrochloric acid comes into contact with potassium permanganate solution?
A: The purple colour of the permanganate disappears.
Source: When hydrochloric acid comes into contact with potassium permanganate, the purple colour of the permanganate disappears.
--------------------------------------------------
Q: What is the purpose of using cubes of different sizes in this demonstration?
A: To show how substances diffuse through a cell and to demonstrate the effect of surface area to volume ratio on diffusion.
Source: The cubes are carefully dropped, at the same time, into a beaker of dilute hydrochloric acid
--------------------------------------------------
Q: Which cube would be the first to turn colourless?
A: The smallest

In [219]:
print(response)

{
  "qa_pairs": [
    {
      "question": "What is agar used for in this demonstration?",
      "answer": "Agar is a jelly that is used to show how substances diffuse through a cell.",
      "source": "Agar is a jelly that is used for growing cultures of bacteria. It has a consistency similar to the cytoplasm of a cell."
    },
    {
      "question": "What happens when hydrochloric acid comes into contact with potassium permanganate?",
      "answer": "The purple colour of the permanganate disappears.",
      "source": "When hydrochloric acid comes into contact with potassium permanganate, the purple colour of the permanganate disappears."
    },
    {
      "question": "What is the purpose of using cubes of different sizes in this demonstration?",
      "answer": "To show how substances diffuse through a cell and to demonstrate the effect of surface area to volume ratio on diffusion.",
      "source": "A Petri dish is prepared which contains a 2 cm deep layer of agar jelly, dyed purp